## Tubo de ensaio
This is a sandbox notebook so I can test out some code and functions.

In [1]:
# Imports
import numpy as np

In [14]:
nums = np.linspace(0,10,11)
nums[::2]

array([ 0.,  2.,  4.,  6.,  8., 10.])

In [7]:
A = np.ones((3,3))
B = (np.array(A)*2)**2
n = np.shape(B)[0]
C = B
C[::n] = 0
D = C+np.transpose(C)/D.max()
print(A,B,C,D)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]] [[0. 0. 0.]
 [4. 4. 4.]
 [4. 4. 4.]] [[0. 0. 0.]
 [4. 4. 4.]
 [4. 4. 4.]] [[0.  0.5 0.5]
 [4.  4.5 4.5]
 [4.  4.5 4.5]]


In [6]:
idx = D>0
D[idx]/np.sum(D[idx])

array([0.08333333, 0.08333333, 0.08333333, 0.16666667, 0.16666667,
       0.08333333, 0.16666667, 0.16666667])

### Test embedding algorithms

In [8]:
# Open results
import pickle

# Select a directory
res_dir = "C:\\Users\\jhflc\\OneDrive\\Documentos\\Projects\\KI_article\\results\\19h38m_16_01_2022\\results_dict.pickle"
file = open(res_dir,'rb')
obj = pickle.load(file)
# Get the expression matrix to use:
exp_matx = obj['Preprocess']['Data']['Exp_matrix']